<a href="https://colab.research.google.com/github/codedm24/Transformers/blob/Transformers-Intro/Tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tokenized text


In [1]:
tokenized_text = "John Doe is  a developer".split()
print(tokenized_text)

['John', 'Doe', 'is', 'a', 'developer']


Loading Tokenizer

In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Using AutoTokenizer

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Using Tokenizsr

In [4]:
tokenizer("Using Hugging Face Tokenizsr is simple")

{'input_ids': [101, 2478, 17662, 2227, 19204, 10993, 21338, 2003, 3722, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Saving a Tokenizer

In [6]:
tokenizer.save_pretrained("C:\Projects\ML\HuggingFace-Colab")

('C:\\Projects\\ML\\HuggingFace-Colab/tokenizer_config.json',
 'C:\\Projects\\ML\\HuggingFace-Colab/special_tokens_map.json',
 'C:\\Projects\\ML\\HuggingFace-Colab/vocab.txt',
 'C:\\Projects\\ML\\HuggingFace-Colab/added_tokens.json',
 'C:\\Projects\\ML\\HuggingFace-Colab/tokenizer.json')

Tokenization

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
sequence = "Using a Mobile Phone is simple"
tokens = tokenizer.tokenize(sequence)
print(tokens)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

['Using', 'a', 'Mobile', 'Phone', 'is', 'simple']
